In [ ]:
#!pip install xgboost lightgbm imblearn mlflow

In [ ]:
#!pip install alibi-detect

In [ ]:
#!pip install shap

In [ ]:
# pip install --upgrade huggingface_hub datasets alibi-detect

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_recall_curve
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import shap
import mlflow
import mlflow.sklearn
from alibi_detect.cd import KSDrift

In [ ]:
df = pd.read_csv("creditcard.csv")

In [ ]:
df

In [ ]:
df['ammount_log'] = np.log(df['Amount'] +1)

In [ ]:
df['ammount_log']

In [ ]:
df['Time_hour'] = df['Time'] //3600
df['Time_day'] = df['Time']//(3600*24)

In [ ]:
df

In [ ]:
for i in range(1,29):
    for j in range(i+1,29):
        df[f'V{i}V{j}'] = df[f'V{i}'] * df[f'V{j}']

In [ ]:
df

In [ ]:
X = df.drop(['Class', "Time"], axis =1)

In [ ]:
y = df['Class']

In [ ]:
X

In [ ]:
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42 )

In [ ]:
sc = StandardScaler()

In [ ]:
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
over = SMOTE(sampling_strategy=0.1, random_state=42)
under = RandomUnderSampler(sampling_strategy=0.5, random_state=42)
steps = [('over', over), ('under', under)]
pipeline = ImbPipeline(steps=steps)
X_resampled, y_resampled = pipeline.fit_resample(X_train, y_train)

In [ ]:
len(X_resampled)

In [ ]:
selector = SelectFromModel(estimator=LGBMClassifier(random_state=42), max_features=50)
X_selected = selector.fit_transform(X_resampled, y_resampled)
selected_features = X.columns[selector.get_support()].tolist()

In [ ]:
len(selected_features)

In [ ]:
pca = PCA(n_components=0.95, random_state=42)
X_pca = pca.fit_transform(X_selected)

In [ ]:
pca.n_components_

In [ ]:
X_pca

In [ ]:
kmeans = KMeans(n_clusters=5, random_state=42)
cluster_labels = kmeans.fit_predict(X_pca)
X_with_clusters = np.column_stack((X_pca, cluster_labels))

In [ ]:
X_with_clusters

In [ ]:
models = {"Random Fortest": RandomForestClassifier(random_state=42),
         "Gradiant Boosting":GradientBoostingClassifier(random_state=42),
         "XGBoost":XGBClassifier(random_state=42),
         "LightBGM": LGBMClassifier(random_state=42),
         "Logistic": LogisticRegression(random_state=42)}

In [ ]:
best_model = None
best_acc = 0

In [ ]:
for name, model in models.items():
    print(f"\nTraining {name}...")
    model.fit(X_with_clusters, y_resampled)
    y_pred = model.predict(np.column_stack((pca.transform(selector.transform(X_test)), kmeans.predict(pca.transform(selector.transform(X_test))))))
    auc = roc_auc_score(y_test, y_pred)
    print(f"AUC for {name}: {auc}")
    if auc > best_acc:
        best_acc = auc
        best_model = model